In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [2]:
import pandas as pd
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")
train = pd.read_csv("../input/nlp-getting-started/train.csv")

In [3]:
train.columns.values, test.columns.values

(array(['id', 'keyword', 'location', 'text', 'target'], dtype=object),
 array(['id', 'keyword', 'location', 'text'], dtype=object))

In [4]:
# train[train['location'].notnull()] # see samples

In [5]:
train['text'].str.extractall(r'(\#)')[0]

      match
0     0        #
3     0        #
4     0        #
      1        #
5     0        #
              ..
7599  1        #
7601  0        #
      1        #
7604  0        #
7607  0        #
Name: 0, Length: 3403, dtype: object

In [6]:
from collections import Counter
from itertools import chain

In [7]:
trial = '#hello World 20 is on Fire #fire #world'

In [8]:
def symbolcount (x,s='#'):
    return len(x.split (s)) - 1

In [9]:
def ulratio (x):
    n = sum(1 for l in x if l.isupper())
    d = sum(1 for l in x if l.islower())
    if d > 0:
        return n/d
    else:
        return 0.999

In [10]:
def nratio (x):
    n = sum(1 for l in x if l.isnumeric())
    d = len(x)
    return n/d

In [11]:
# train['text'].apply (symbolcount, args=('@'))

In [12]:
from nltk.corpus import stopwords
import nltk
import re

nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
def clean_text(x):
    x = x.lower()
    x = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x)
    stop_words = stopwords.words ('english')
    words = re.sub(r'[^\w\s]','',x).lower().split(' ')
    words = [word for word in words if word not in stop_words]
    return words

In [14]:
# clean_text(train['text'][1])

In [15]:
def transformer (x):
    assert isinstance(x, pd.DataFrame), 'Input (x) not a DataFrame'
    
    x['text_clean'] = x['text'].apply(clean_text)
    x['ratio'] = x['text'].apply(ulratio)
    x['nratio'] = x['text'].apply(nratio)
    x['hashcount'] = x['text'].apply(symbolcount, args=('#'))
    x['atcount'] = x['text'].apply(symbolcount, args=('@'))
    x['length'] = x['text'].apply(len)/280
    x['key'] = ~x['keyword'].isna()
    x['geo'] = ~x['location'].isna()
    x['key'] = x['key'] * 1
    x['geo'] = x['geo'] * 1
    
    return x

In [16]:
train = transformer (train)
test = transformer (test)

In [17]:
r_train = train

In [18]:
# r_train['target'] = revtarget (r_train)

In [19]:
train.head()

,id,keyword,location,text,target,text_clean,ratio,nratio,hashcount,atcount,length,key,geo
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[deeds, reason, earthquake, may, allah, forgiv...",0.217391,0.000000,1,0,0.246429,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]",0.192308,0.000000,0,0,0.135714,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[residents, asked, shelter, place, notified, o...",0.018692,0.000000,0,0,0.475000,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[13000, people, receive, wildfires, evacuation...",0.020408,0.076923,1,0,0.232143,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[got, sent, photo, ruby, alaska, smoke, wildfi...",0.044776,0.000000,2,0,0.314286,0,0


In [20]:
r_train.to_csv(path_or_buf='train_r.csv', index=False)
test.to_csv(path_or_buf='test.csv', index=False)
train.to_csv(path_or_buf='train.csv', index=False)

In [21]:
# train[['text_clean','ratio', 'nratio', 'hashcount', 'atcount', 'length', 'key', 'geo']].values

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
max_vocab_size = 60000
tweet = 280


tokenizer = Tokenizer (num_words = max_vocab_size)
tokenizer.fit_on_texts (train['text_clean'].values)

sequence_train = tokenizer.texts_to_sequences (train['text_clean'])
sequence_train = pad_sequences (sequence_train, maxlen=tweet, padding='post')
sequence_test = tokenizer.texts_to_sequences (test['text_clean'])
sequence_test = pad_sequences (sequence_test, maxlen=tweet, padding='post')

In [24]:
meta_train = train[['ratio', 'nratio', 'hashcount', 'atcount', 'length', 'key', 'geo']].values
meta_test = test[['ratio', 'nratio', 'hashcount', 'atcount', 'length', 'key', 'geo']].values

In [25]:
meta_train.shape, sequence_train.shape

((7613, 7), (7613, 280))

In [26]:
X_train =  np.concatenate((sequence_train,meta_train),axis=1)
X_test =  np.concatenate((sequence_test,meta_test),axis=1)
y_train = train['target'].values

In [27]:
Tt = sequence_train.shape[1]
Tm = meta_train.shape[1]

### sklearn
*https://towardsdatascience.com/how-to-enter-your-first-kaggle-competition-4717e7b232db*

In [28]:
def join_words (x):
    return ' '.join(x)

In [29]:
train_sgd = train['text_clean'].apply(join_words)
test_sgd = test['text_clean'].apply(join_words)

train_target_sgd = train['target']

In [30]:
train_sgd.shape, train_target_sgd.shape

((7613,), (7613,))

In [31]:
from sklearn.model_selection import train_test_split

X_train_sgd, X_test_sgd, y_train_sgd, y_test_sgd = train_test_split(train_sgd, train_target_sgd, random_state = 23)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier

pipeline_xgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('xgc', XGBClassifier()),
])

model_xgb = pipeline_xgd.fit(X_train_sgd, y_train_sgd)

In [32]:
predict_xgb = model_xgb.predict_proba (test_sgd)

In [33]:
predict_xgb = predict_xgb[:,1]

In [34]:
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator = model_xgb, X = X_train_sgd, y = y_train_sgd, cv = 10)
accuracies.mean(), accuracies.std()

(0.7185172212492703, 0.016939140040076094)

### lemmatizer

In [35]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
def lemma (x):
    return [lemmatizer.lemmatize (word) for word in x if not word.isnumeric()]

In [37]:
train_lemma = train['text_clean'].apply (lemma)
test_lemma = test['text_clean'].apply (lemma)

In [38]:
max_vocab_size_l = 40000

tokenizer_l = Tokenizer (num_words = max_vocab_size_l)
tokenizer_l.fit_on_texts (train_lemma.values)

sequence_train_lemma = tokenizer.texts_to_sequences (train_lemma)
sequence_train_lemma = pad_sequences (sequence_train, maxlen=tweet, padding='post')
sequence_test_lemma = tokenizer.texts_to_sequences (test_lemma)
sequence_test_lemma = pad_sequences (sequence_test, maxlen=tweet, padding='post')

In [39]:
Tl = sequence_train_lemma.shape[1]

### tensorflow

In [40]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Reshape, Embedding, LSTM, GlobalMaxPooling1D, concatenate, MaxPooling1D, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

In [41]:
word_to_idx = tokenizer.word_index
V = len (word_to_idx) # (V)ocab

In [42]:
word_to_idx = tokenizer_l.word_index
Vl = len (word_to_idx)

In [43]:
D = 64 # embedding dimension

input_t = Input (shape=(Tt,))
input_l = Input (shape=(Tl,))
input_m = Input (shape=(Tm,))


w = Embedding (V+1, D) (input_l)
w = Dropout (0.5) (w)
w = Bidirectional (LSTM (64, return_sequences=True)) (w)
w = BatchNormalization () (w)
w = GlobalMaxPooling1D () (w)
w = Dropout (0.5) (w)
w = Dense (32, activation='relu') (w)
w = Model (inputs=input_l, outputs=w)

x = Embedding (V+1, D) (input_t)
x = Dropout (0.5) (x)
x = Bidirectional (LSTM (128, return_sequences=True)) (x)
x = BatchNormalization () (x)
x = GlobalMaxPooling1D () (x)
x = Dropout (0.5) (x)
x = Dense (64, activation='relu') (x)
x = Model (inputs=input_t, outputs=x)

y = Dense (32, activation="relu") (input_m)
y = Dropout (0.5) (y)
y = Dense (8, activation="relu") (y)
y = Model (inputs=input_m, outputs=y)

combined = concatenate([x.output, w.output, y.output])

z = Dense (96, activation="relu") (combined)
z = Dropout (0.5) (z)
z = Dense (32, activation="relu") (z)
z = Dense (1, activation="sigmoid") (z)


In [44]:
model = Model(inputs=[x.input, w.input, y.input], outputs=z)

In [45]:
adam = Adam (learning_rate=0.005, decay=5e-6)

In [46]:
model.compile (optimizer=adam,
               loss='binary_crossentropy',
               metrics=['accuracy'])

In [47]:
y_train.shape

(7613,)

In [48]:
epochs = 24

r = model.fit ([sequence_train, sequence_train_lemma, meta_train], y_train, epochs=epochs, batch_size=64)

Train on 7613 samples
Epoch 1/24
7613/7613 [==============================] - 19s 2ms/sample - loss: 0.6653 - accuracy: 0.5952
Epoch 2/24
7613/7613 [==============================] - 10s 1ms/sample - loss: 0.4422 - accuracy: 0.8189
Epoch 3/24
7613/7613 [==============================] - 9s 1ms/sample - loss: 0.2815 - accuracy: 0.9031
Epoch 4/24
7613/7613 [==============================] - 9s 1ms/sample - loss: 0.1811 - accuracy: 0.9345
Epoch 5/24
7613/7613 [==============================] - 9s 1ms/sample - loss: 0.1379 - accuracy: 0.9506
Epoch 6/24
7613/7613 [==============================] - 9s 1ms/sample - loss: 0.1106 - accuracy: 0.9590
Epoch 7/24
7613/7613 [==============================] - 9s 1ms/sample - loss: 0.0960 - accuracy: 0.9614
Epoch 8/24
7613/7613 [==============================] - 10s 1ms/sample - loss: 0.0798 - accuracy: 0.9679
Epoch 9/24
7613/7613 [==============================] - 9s 1ms/sample - loss: 0.0725 - accuracy: 0.9703
Epoch 10/24
7613/7613 [================

In [49]:
predict_tf = model.predict ([sequence_test, sequence_test_lemma, meta_test])

In [50]:
test['target'] = np.round (predict_tf)
test['target'] = test['target'].astype(int)

In [51]:
test[['id','target']].to_csv(path_or_buf='result.csv', index=False)

In [52]:
predict_combo = np.add ((predict_xgb.reshape(-1,1)*.1), (predict_tf*.9))

In [53]:
test['target'] = np.round (predict_combo)
test['target'] = test['target'].astype(int)

In [54]:
test[['id','target']].to_csv(path_or_buf='result_combo.csv', index=False)

In [55]:
test['target'] = np.round (predict_xgb.reshape(-1,1))
test['target'] = test['target'].astype(int)

In [56]:
test[['id','target']].to_csv(path_or_buf='result_xgb.csv', index=False)